In [ ]:
# ------------------------------
# Whisper Batch Transcriber for MP4 Videos
# ------------------------------

import os
import csv
import whisper
import torch

# ---------------- Settings ----------------
LANGUAGE = "es"
MODEL_SIZE = "medium"
INPUT_DIR = r"C:\Users\micha\spanish_articulation\h"
OUTPUT_CSV = "transcriptions.csv"

# ---------------- Device detection ----------------
if torch.cuda.is_available():
    DEVICE = "cuda"
    print(f"⚡ Using GPU: {torch.cuda.get_device_name(0)}")
else:
    DEVICE = "cpu"
    print("🖥️ No GPU detected, using CPU.")

print("Loading Whisper model...")
model = whisper.load_model(MODEL_SIZE, device=DEVICE)
print("✅ Model loaded!")

# ---------------- Collect MP4 files ----------------
files = [f for f in os.listdir(INPUT_DIR) if f.lower().endswith(".mp4")]
files.sort()  # alphabetical order

print(f"Found {len(files)} video files.")

# ---------------- Transcribe and write CSV ----------------
with open(OUTPUT_CSV, mode="w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["filename", "text"])

    for fname in files:
        path = os.path.join(INPUT_DIR, fname)
        print(f"🎬 Transcribing {fname} ...")

        try:
            result = model.transcribe(path, language=LANGUAGE, task="transcribe")
            text = result.get("text", "").strip()
            writer.writerow([fname, text])
            csvfile.flush()   # <-- force write to disk
            print(f"✅ Done: {fname}")
        except Exception as e:
            print(f"❌ Error transcribing {fname}: {e}")
            writer.writerow([fname, f"ERROR: {e}"])
            csvfile.flush()


print(f"📄 All transcriptions written to {OUTPUT_CSV}")


⚡ Using GPU: NVIDIA T1000 8GB
Loading Whisper model...
✅ Model loaded!
Found 163 video files.
🎬 Transcribing 1_adher_front_neg.mp4 ...
✅ Done: 1_adher_front_neg.mp4
🎬 Transcribing 1_adher_side_eg.mp4 ...
✅ Done: 1_adher_side_eg.mp4
🎬 Transcribing 1_adher_side_neg.mp4 ...
✅ Done: 1_adher_side_neg.mp4
🎬 Transcribing 1_ah_front_eg.mp4 ...
✅ Done: 1_ah_front_eg.mp4
🎬 Transcribing 1_ah_front_neg.mp4 ...
✅ Done: 1_ah_front_neg.mp4
🎬 Transcribing 1_ah_side_eg.mp4 ...
✅ Done: 1_ah_side_eg.mp4
🎬 Transcribing 1_ah_side_neg.mp4 ...
✅ Done: 1_ah_side_neg.mp4
🎬 Transcribing 1_ahora_front_eg.mp4 ...
✅ Done: 1_ahora_front_eg.mp4
🎬 Transcribing 1_ahora_front_neg.mp4 ...
✅ Done: 1_ahora_front_neg.mp4
🎬 Transcribing 1_ahora_side_eg.mp4 ...
✅ Done: 1_ahora_side_eg.mp4
🎬 Transcribing 1_ahora_side_neg.mp4 ...
✅ Done: 1_ahora_side_neg.mp4
🎬 Transcribing 1_ahumado_front_eg.mp4 ...
✅ Done: 1_ahumado_front_eg.mp4
🎬 Transcribing 1_ahumado_front_neg.mp4 ...
✅ Done: 1_ahumado_front_neg.mp4
🎬 Transcribing 1_ahumad

In [ ]:
# ------------------------------
# Whisper Batch Transcriber for MP4 Videos + Duration & Speed
# ------------------------------

import os
import csv
import whisper
import torch
from pydub import AudioSegment
import time

# ---------------- Settings ----------------
LANGUAGE = "es"
MODEL_SIZE = "medium"
INPUT_DIR = r"C:\Users\micha\spanish_articulation\h"
OUTPUT_CSV = "transcriptions.csv"

# ---------------- Device detection ----------------
if torch.cuda.is_available():
    DEVICE = "cuda"
    print(f"⚡ Using GPU: {torch.cuda.get_device_name(0)}")
else:
    DEVICE = "cpu"
    print("🖥️ No GPU detected, using CPU.")

print("Loading Whisper model...")
model = whisper.load_model(MODEL_SIZE, device=DEVICE)
print("✅ Model loaded!")

# ---------------- Collect MP4 files ----------------
files = [f for f in os.listdir(INPUT_DIR) if f.lower().endswith(".mp4")]
files.sort()  # alphabetical order

print(f"Found {len(files)} video files.")

# ---------------- Transcribe and write CSV ----------------
with open(OUTPUT_CSV, mode="w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["filename", "text", "audio_duration_sec", "transcription_time_sec"])

    for fname in files:
        path = os.path.join(INPUT_DIR, fname)
        print(f"\n🎬 Transcribing {fname} ...")

        try:
            # Load audio track from video
            audio = AudioSegment.from_file(path, format="mp4")
            duration_sec = len(audio) / 1000.0

            print(f"   ▶ Audio duration: {duration_sec:.2f} sec")

            # Time the transcription
            start_time = time.time()
            result = model.transcribe(path, language=LANGUAGE, task="transcribe")
            end_time = time.time()
            transcription_time = end_time - start_time

            text = result.get("text", "").strip()

            # Write to CSV
            writer.writerow([fname, text, f"{duration_sec:.2f}", f"{transcription_time:.2f}"])
            csvfile.flush()

            # Print transcription snippet and timing
            print(f"✅ Done: {fname}")
            print(f"   Transcript: {text[:120]}{'...' if len(text) > 120 else ''}")
            print(f"   ▶ Transcription time: {transcription_time:.2f} sec")
            if duration_sec > 0:
                speed_ratio = duration_sec / transcription_time
                print(f"   ▶ Speed ratio: {speed_ratio:.2f}x (audio length / transcription time)")

        except Exception as e:
            print(f"❌ Error transcribing {fname}: {e}")
            writer.writerow([fname, f"ERROR: {e}", "", ""])
            csvfile.flush()

print(f"\n📄 All transcriptions written to {OUTPUT_CSV}")
